In [ ]:
# Remember to install tianshou first
!pip install tianshou==0.4.8
!pip install gym

# Overview
From its literal meaning, we can easily know that the Collector in Tianshou is used to collect training data. More specificly, the Collector controls the interaction between Policy (agent) and the environment. It also helps save the interaction data into the ReplayBuffer and returns episode statistics.

<center>
</img>
</center>



# Usages
Collector can be used both for training (data collecting) and evaluation in Tianshou.

## Policy evaluation
We need to evaluate our trained policy from time to time in DRL experiments. Collector can help us with this.

First we have to initialise a Collector with an (vectorized) environment and a given policy (agent).

In [ ]:
import gym
import numpy as np
import torch

from tianshou.data import Collector
from tianshou.env import DummyVectorEnv
from tianshou.policy import PGPolicy
from tianshou.utils.net.common import Net
from tianshou.utils.net.discrete import Actor

import warnings
warnings.filterwarnings('ignore')

env = gym.make("CartPole-v0")
test_envs = DummyVectorEnv([lambda: gym.make("CartPole-v0") for _ in range(2)])

# model
net = Net(env.observation_space.shape, hidden_sizes=[16,])
actor = Actor(net, env.action_space.shape)
optim = torch.optim.Adam(actor.parameters(), lr=0.0003)

policy = PGPolicy(actor, optim, dist_fn=torch.distributions.Categorical)
test_collector = Collector(policy, test_envs)

Now we would like to collect 9 episodes of data to test how our initialised Policy performs.

In [ ]:
collect_result = test_collector.collect(n_episode=9)
print(collect_result)
print("Rewards of 9 episodes are {}".format(collect_result["rews"]))
print("Average episode reward is {}.".format(collect_result["rew"]))
print("Average episode length is {}.".format(collect_result["len"]))

{'n/ep': 9, 'n/st': 85, 'rews': array([ 9.,  9., 10., 10.,  9.,  9., 10., 10.,  9.]), 'lens': array([ 9,  9, 10, 10,  9,  9, 10, 10,  9]), 'idxs': array([0, 1, 0, 1, 0, 1, 0, 1, 1]), 'rew': 9.444444444444445, 'len': 9.444444444444445, 'rew_std': 0.49690399499995325, 'len_std': 0.49690399499995325}
Rewards of 9 episodes are [ 9.  9. 10. 10.  9.  9. 10. 10.  9.]
Average episode reward is 9.444444444444445.
Average episode length is 9.444444444444445.


Now we wonder what is the performance of a random policy.

In [ ]:
# Reset the collector
test_collector.reset()
collect_result = test_collector.collect(n_episode=9, random=True)
print(collect_result)
print("Rewards of 9 episodes are {}".format(collect_result["rews"]))
print("Average episode reward is {}.".format(collect_result["rew"]))
print("Average episode length is {}.".format(collect_result["len"]))

{'n/ep': 9, 'n/st': 187, 'rews': array([13., 14., 11., 44., 35., 14., 19., 17., 20.]), 'lens': array([13, 14, 11, 44, 35, 14, 19, 17, 20]), 'idxs': array([1, 0, 0, 1, 0, 0, 1, 0, 1]), 'rew': 20.77777777777778, 'len': 20.77777777777778, 'rew_std': 10.580671872993257, 'len_std': 10.580671872993257}
Rewards of 9 episodes are [13. 14. 11. 44. 35. 14. 19. 17. 20.]
Average episode reward is 20.77777777777778.
Average episode length is 20.77777777777778.


Seems that an initialised policy performs even worse than a random policy without any training.

## Data Collecting
Data collecting is mostly used during training, when we need to store the collected data in a ReplayBuffer.

In [ ]:
from tianshou.data import VectorReplayBuffer
train_env_num = 4
buffer_size = 100
train_envs = DummyVectorEnv([lambda: gym.make("CartPole-v0") for _ in range(train_env_num)])
replaybuffer = VectorReplayBuffer(buffer_size, train_env_num)

train_collector = Collector(policy, train_envs, replaybuffer)

Now we can collect 50 steps of data, which will be automatically saved in the replay buffer. You can still choose to collect a certain number of episodes rather than steps. Try it yourself.

In [ ]:
print(len(replaybuffer))
collect_result = train_collector.collect(n_step=50)
print(len(replaybuffer))
print(collect_result)

0
52
{'n/ep': 4, 'n/st': 52, 'rews': array([ 8., 10., 10., 10.]), 'lens': array([ 8, 10, 10, 10]), 'idxs': array([25,  0, 50, 75]), 'rew': 9.5, 'len': 9.5, 'rew_std': 0.8660254037844386, 'len_std': 0.8660254037844386}


In [ ]:
for i in range(13):
    print(i, replaybuffer.next(i))

0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 9
10 11
11 12
12 12


In [ ]:
replaybuffer.sample(10)

(Batch(
     obs: array([[-6.36541036e-02, -1.15297838e+00,  7.85138179e-02,
                   1.77225357e+00],
                 [-1.05090645e-02, -3.71521519e-01, -2.96323181e-03,
                   5.76793524e-01],
                 [-2.45282997e-02,  4.77850180e-02,  2.21142716e-02,
                   4.96743371e-02],
                 [ 1.68433453e-03,  4.47272356e-02, -1.72360346e-02,
                  -7.74977680e-03],
                 [ 4.29854159e-02, -3.95380051e-01,  3.91005958e-02,
                   6.41183774e-01],
                 [ 3.50778149e-02, -5.91024637e-01,  5.19242712e-02,
                   9.45918993e-01],
                 [-1.19358173e-01, -1.38179912e+00,  1.04318690e-01,
                   2.10185768e+00],
                 [-3.48454722e-02,  1.61145106e-03, -2.49944951e-02,
                  -3.82213155e-02],
                 [-3.48454722e-02,  1.61145106e-03, -2.49944951e-02,
                  -3.82213155e-02],
                 [-4.64338716e-02, -5.82710950e

# Further Reading
The above collector actually collects 52 data at a time because 52 % 4 = 0. There is one asynchronous collector which allows you collect exactly 50 steps. Check the [documentation](https://tianshou.readthedocs.io/en/master/api/tianshou.data.html#asynccollector) for details.